# Tropical Cyclone Detection

References:
- https://gmd.copernicus.org/articles/10/1069/2017/
- https://gmd.copernicus.org/articles/14/5023/2021/
- https://www.nhc.noaa.gov/data/tcr/AL172020_Paulette.pdf

# Detector Plugin
Write the plugin code here: [detectors/hurricane/hurricane_detector.py](../detectors/hurricane/hurricane_detector.py)

The simplest approach is to start by detecting the minimum pressure. To improve the detection the following steps can be used which are also described in the papers above. The described steps in the papers can be used for a more advanced implementation which would do the following:
1. Find local minima in the sea-level pressure (field: `pres_msl`). This is a candidate point.
   - you need to perform an `MPI_Allreduce` operation here
2. For every found local minimum up to a certain threshold (like $10100$) do the following:
  1. Check the closed contour criteria for pressure (see [tools/polygon.py](../tools/polygon.py))
  2. Interpolate the geopotential (ICON field: `geopot`) onto $300$ hPa and $500$ hPa.
  3. find the maximum geopotential around the center around 1° gcd on both surfaces
  4. check the contour criteria on both surfaces (see [tools/polygon.py](../tools/polygon.py))
  5. write some output with data around the center. You can use the `write_output_zarr.py` tool for that)

# Run the experiment

- Master namelist: [icon_master.namelist](../experiments/hurricane_paulette/icon_master.namelist)
    - set `lrestart` to `.TRUE.` to start from an restart file
    - the restart file can be set by linking the restart file like this. Restart files are available every 8 hours:

In [ ]:
!ln -sfn /work/bk1414/summerschool/hurricane_paulette_restart_files/restart_atm_20200913T160000Z.mfr ../experiments/hurricane_paulette/multifile_restart_atm.mfr

- Atmo namelist: [NAMELIST_Paulette_tc](../experiments/hurricane_paulette/NAMELIST_Paulette_tc)
  - in the `comin_nml` you can configure the plugin (already done)
  - in `run_nml` set `nsteps` to the number of steps you want to execute (or to 0 to run until the experiment end)

To run the experiment and test the plugin execute the following cell.

In [ ]:
from IPython.display import display, HTML
jid=!env -i sbatch --parsable --chdir=../experiments/hurricane_paulette ../experiments/hurricane_paulette/run.sh
log_file=f"../experiments/hurricane_paulette/logs/LOG.hurricane_paulette.run.{jid[0]}.o"
display(HTML(f"Started job {jid[0]} with logfile: <a href='{log_file}' >{log_file}</a>"))

You can follow the jobs scheduling with the following command

In [ ]:
!squeue --me

And observe the output with this command:

In [ ]:
!tail -n+1 -F $log_file

If you write the output with the `write_output_zarr` tool into a `output.zarr` you can visualize the dataset like this:

In [ ]:
import uxarray as ux
import cartopy.crs as ccrs
import geoviews.feature as gf
import numpy as np
background = gf.coastline(projection=ccrs.PlateCarree()) * gf.ocean(projection=ccrs.PlateCarree()) * gf.land(projection=ccrs.PlateCarree())
ds = ux.open_dataset("../experiments/hurricane_paulette/domain1_DOM01.nc", "../experiments/hurricane_paulette/output.zarr")
t = np.datetime64("2020-09-13T16:10:00")
ds

In [ ]:
 background * ds["pres_msl"].sel(time=t).plot(projection=ccrs.PlateCarree())

In [ ]:
 background * ds["sp_10m"].sel(time=np.datetime64("2020-09-13T16:10:00")).plot(projection=ccrs.PlateCarree())